In [1]:
!git clone https://github.com/NSCC-COGS/Aestheta.git

Cloning into 'Aestheta'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 579 (delta 106), reused 65 (delta 29), pack-reused 389
Receiving objects: 100% (579/579), 7.48 MiB | 6.09 MiB/s, done.
Resolving deltas: 100% (310/310), done.


In [2]:
%cd /content/Aestheta/
!git pull

/content/Aestheta
Already up to date.


In [3]:
import Aestheta.Library.core as core
earth_RGB = core.getTile(source='google_sat')

0 0 0


In [ ]:
core.getTile(source='google_sat', show=True)

In [ ]:
core.norm_diff(earth_RGB,  B1=1, B2=2, show=True)
core.norm_diff(earth_RGB,  B1=2, B2=3, show=True)
core.norm_diff(earth_RGB,  B1=3, B2=1, show=True)

In [ ]:
from matplotlib import pyplot as plt
for i in range(3):
  plt.imshow(earth_RGB[:,:,i])
  plt.show()

In [7]:
# ND1 = core.norm_diff(earth_RGB,  B1=1, B2=2, show=False)
# ND2 = core.norm_diff(earth_RGB,  B1=2, B2=3, show=False)
# ND3 = core.norm_diff(earth_RGB,  B1=3, B2=1, show=False)

In [8]:
def NDPI(img, show=False):
  import numpy as np
  ND1 = core.norm_diff(img,  B1=1, B2=2, show=False)
  ND2 = core.norm_diff(img,  B1=2, B2=3, show=False)
  ND3 = core.norm_diff(img,  B1=3, B2=1, show=False)

  NDPI = np.dstack([ND1,ND2,ND3])
  
  # NDPI-=np.nanmin(NDPI)
  # NDPI/=np.nanmax(NDPI)

  NDPI -= -.5
  NDPI /= 1.2
  NDPI*=255

  NDPI = np.uint8(NDPI)
  
  print(NDPI.min(),NDPI.max())
  if show:
    from matplotlib import pyplot as plt
    plt.imshow(NDPI)
    plt.show()
    return NDPI
  else:
    return NDPI

In [ ]:
earth_NDPI = NDPI(earth_RGB, show=True)

In [10]:
# kmeans cluster
# more info at
# https://www.thepythoncode.com/article/kmeans-for-image-segmentation-opencv-python

def kmeans(image, k=3, show = True):

  import cv2
  import numpy as np
  import matplotlib.pyplot as plt

  pixel_values = image.reshape((-1, 3))
  pixel_values = np.float32(pixel_values)

  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
  _, labels, (centers) = cv2.kmeans(pixel_values, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

  centers = np.uint8(centers)
  labels = labels.flatten()
  segmented_image = centers[labels.flatten()]
  segmented_image = segmented_image.reshape(image.shape)

  if show:
    plt.imshow(segmented_image)
    plt.show()
    return segmented_image
  else:
    return segmented_image

In [ ]:
earth_k3= kmeans(earth_RGB, k=3,show=True)

In [ ]:
earth_NDPI_k3 = kmeans(earth_NDPI, k=3,show=True)

In [ ]:
earth_k7= kmeans(earth_RGB, k=7,show=True)
earth_NDPI_k7 = kmeans(earth_NDPI, k=7,show=True)

In [16]:
def unique_colors(image, show=True):
  import numpy as np

  classes,arr_classes =  np.unique(image.reshape(-1, image.shape[2]), axis=0, return_inverse=True)
  img_classes = np.reshape(arr_classes,image[:,:,0].shape)


  if show:
    plt.imshow(img_classes, cmap='rainbow')
    plt.show()
    return img_classes

  else:
    return img_classes


In [ ]:
earth_k7_class = unique_colors(earth_k7)
earth_NDPI_k7_class = unique_colors(earth_NDPI_k7)

In [20]:
def edge(image, show=True, abs=True, mask=True):
  import numpy as np
  edges = core.image_shift_diff(image, axis=0)
  edges += core.image_shift_diff(image, axis=1)

  if abs:
    edges = np.abs(edges)

  if mask:
    edges = edges==0

  if show:
    # plt.imshow(np.mod(edges,2), cmap='viridis')
    plt.imshow(edges, cmap='viridis', interpolation='Nearest')
    plt.show()
    return edges

  else:
    return edges

In [ ]:
earth_k7_class_seg = edge(earth_k7_class)
earth_NDPI_k7_class_seg = edge(earth_NDPI_k7_class)

In [22]:
def segment_old(image, show=True):

  edges = edge(image, show=False)
  from scipy.ndimage import label
  # thresholdImage = earth_NDPI_k3 > earth_NDPI_k3.mean()
  # img_segments=np.mod(image,3)
  img_segments,unused = label(edges)

  # img_segments[~edges] -= core.image_shift_diff(img_segments, axis=0)[~edges]

  edges = edge(img_segments, show=False)

  # edges = core.image_shift_diff(image, axis=0)
  # edges += core.image_shift_diff(image, axis=1)

  # for i in range(0,image.max()):
  #   print

  # print(image.max(), image.min())

  # img_segments*=np.random.randint(0,2,size=img_segments.shape)

  # from scipy import stats
  # print(stats.describe(img_segments))

  # import numpy as np
  # labels, pixelCount = np.unique(img_segments, return_counts=True)
  # sorter = np.argsort(pixelCount)
  # img_segments = sorter[img_segments]

  # img_segments, pixelCount = np.unique(img_segments, return_counts=True)
  # img_segments = np.reshape(img_segments,image)
  #img_segments = unique_colors(img_segments,show=False)
  print(unused)
  # print(labelMask.max())
  # print(labelMask.min())
  if show:
    # plt.imshow(np.mod(img_segments,4), cmap='viridis')
    plt.imshow(img_segments, cmap='viridis')
    # plt.imshow(edges, cmap='viridis')
    plt.show()
    return img_segments

  else:
    return img_segments

In [23]:
def segment(image, show=True, sort = 'random'):

  import numpy as np
  from scipy.ndimage import label

  img_segments = np.zeros_like(image)

  uniques = np.unique(image)

  for i in uniques:
    # print(i)
    i_segments,unused = label(image==i)
    i_segments[i_segments<0]+=img_segments.max()
    # print(img_segments.min(), img_segments.max())
    img_segments+=i_segments

  print('nsegments', img_segments.max())

  if sort == 'random':
    labels = np.arange(img_segments.max())
    seed = np.random.rand(img_segments.max())
    sorter = np.argsort(seed)
    labels = sorter[labels-1]
    img_segments = labels[img_segments-1]

  if sort == 'size':
    '''currently does not seem to be qworking correctly'''
    labels, pixelCount = np.unique(img_segments, return_counts=True)
    print(labels.shape)
    sorter = np.argsort(pixelCount)
    print(labels)
    labels = sorter[labels-1]
    labels+=1
    print(labels)
    test = np.arange(0,100)
    test = labels[test]
    print(img_segments.max())
    img_segments = labels[img_segments-1]

  if show:
    # plt.imshow(np.mod(img_segments,4), cmap='viridis')
    plt.imshow(img_segments, cmap='magma')
    # plt.imshow(edges, cmap='viridis')
    plt.show()
    return img_segments

  else:
    return img_segments

In [ ]:
earth_k7_class_seg = segment(earth_k7_class)
earth_NDPI_k7_class_seg = segment(earth_NDPI_k7_class)

In [25]:
def RGB_v_NDPI(location=None, k=3, image=False):
  if location:
    img_RGB = core.getTile(xyz = location, source='google_sat')
  else:
    img_RGB = image
  RGB_k = kmeans(img_RGB, k=3,show=True)
  RGB_k_class = unique_colors(RGB_k,show=True)
  img_NDPI = NDPI(img_RGB, show=False)
  NDPI_k = kmeans(img_NDPI, k=3,show=True)
  NDPI_k_class = unique_colors(NDPI_k,show=True)

In [ ]:
ns = [-65.0,45.5,5]
RGB_v_NDPI(ns)

In [ ]:
city = [-63.5752,44.6488,17]
RGB_v_NDPI(city)

In [ ]:
forest = [-64.937823,44.902670,16]
RGB_v_NDPI(forest)

In [ ]:
shore = [-64.690149,46.449160,19]
RGB_v_NDPI(shore)

In [ ]:
bay = [-64.721486,46.464353,15]
RGB_v_NDPI(bay)

Lets look at Zostera Data!

In [ ]:
%cd /content/
!wget -O PQ_RCD_50cm_Cubic.tif https://nscc-my.sharepoint.com/:i:/g/personal/w0001006_campus_nscc_ca/EeGmrgtNc9ZErvlXcVXfNaABsgdvFf7XTX-RprngA9KaCA?download=1

In [32]:
import imageio
b = '/content/PQ_RCD_50cm_Cubic.tif'
a = imageio.imread(b)
print(a.shape)

(21450, 21710, 4)


In [ ]:
xWater= int(13000)
yWater = int(12500)
# xWater= int(13200)
# yWater = int(12500)
aWater_RGBN = a[xWater:xWater+256,yWater:yWater+256,:]
aWater_RGB = aWater_RGBN[:,:,0:3]

RGB_v_NDPI(image = aWater_RGB, k=3)

Lets grab some lansat!!

In [34]:
%cd /content/
!git clone https://github.com/NSCC-COGS/GDAA2030.git

/content
Cloning into 'GDAA2030'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 489 (delta 56), reused 19 (delta 9), pack-reused 370
Receiving objects: 100% (489/489), 65.92 MiB | 18.97 MiB/s, done.
Resolving deltas: 100% (214/214), done.


In [35]:
%cd /content/GDAA2030/tutorial2/kevinkmcguigan
import GDAA2030_T2_kevinmc_getLandsat_functions as glf

/content/GDAA2030/tutorial2/kevinkmcguigan


In [36]:
def convert32(a, maskVal=0, inplace=False):
  if inplace:
    b=a
  else:
    b=a.copy()
  mask = b == maskVal
  b = b.astype(np.float32)
  b[mask] = np.nan
  return b

In [37]:
def stdScale(a, n=1, inplace=False):
  if inplace:
    b=a
  else:
    b=a.copy()
  s0 = np.nanmean(b) - np.nanstd(b)/n
  s1 = np.nanmean(b) + np.nanstd(b)/n
  print('Scale from %f to %f' %(s0,s1))
  b[b<s0] = s1 # where the arrays is LESS than the mean - std, SET it to the mean - std
  b[b>s1] = s1# where the arrays is GREATER than the mean + std, SET it to the mean + std
  return b

In [38]:
def normalize(a, inplace=False):
  if inplace:
    b=a
  else:
    b=a.copy()

  if b.dtype != np.float32:
    b = b.astype(np.float32)

  b -= np.nanmin(b)
  b /= np.nanmax(b)
  # print(np.nanmin(b), np.nanmax(b))

  return b


In [39]:
def scale8bit(a, return_mask=True, inplace=False):
  if inplace:
    b=a
  else:
    b=a.copy()

  if return_mask:
    mask = ~np.isnan(b)
  b *= 255
  b = b.astype(np.uint8)

  if return_mask:
    mask = mask.astype(np.uint8)
    mask *= 255

    return b,mask

  else:
    return b

In [40]:
def ourPlot(a, interpolation = 'bilinear', histogram=True):
  stats = {
      'max' : np.nanmax(a),
      'min' : np.nanmin(a),
      'mean' : np.nanmean(a),
      'std' : np.nanstd(a),
      'bitDepth' : a.dtype,
      'dimensions' : a.shape,
      'top_left_value' : a[0,0]
  }

  for item in stats:
    print('%s: %s'%(item, stats[item]))

  plt.cla()
  plt.subplot(121)
  plt.imshow(a ,interpolation = interpolation)
  if histogram:
    plt.subplot(122)
    plt.hist(a.flatten(),bins=100)
    s0 = stats['mean'] - stats['std']
    s1 = stats['mean'] + stats['std']
    plt.axvline(s0,c='red')
    plt.axvline(s1,c='red')

In [44]:
def makeCompositeRGBA(bands=[4,3,2]):
  import tifffile
  images = []
  for band in bands:
    imageUrl, imagePath = glf.getImageIO(selectedScene, band, imageDir)
    glf.downloadImage(imageUrl,imagePath)
    im = tifffile.imread(imagePath)[::10,::10] # this will reduce the size of the arrays by a factor of 10x10 to save on memory
    im = convert32(im, inplace=True)
    im = stdScale(im, n=.5, inplace=True) # note that I did play with the number of standard deviation here
    im = normalize(im, inplace=True)
    im,mask = scale8bit(im, return_mask=True, inplace=True)
    images.append(im)
  images.append(mask)

  return np.dstack(images)

In [ ]:
lat = 44.88
lon = -65.16
band = 1
cloudCoverMax = 2
# sceneList = "scene_list.txt"
sceneList = 'https://landsat-pds.s3.amazonaws.com/c1/L8/scene_list.gz'
sceneTest = None # set a small number to test, or None
imageDir = r'.\images'

sceneDf = glf.loadSceneList(sceneList, sceneTest, cloudCoverMax)

# sceneDf = glf.loadSceneList(sceneList)

path,row = glf.findPathRow(sceneDf, lat, lon)

# print(path, row)

selectedScene = glf.selectScene(sceneDf, path, row)

In [ ]:
import numpy as np
landsat_RGBA = makeCompositeRGBA([4,3,2])
ourPlot(landsat_RGBA)

In [ ]:
landsat_RGB = landsat_RGBA[:,:,0:3]
RGB_v_NDPI(image = landsat_RGB,k=3)